# Getting Song Lyrics

from Genius

for dr tracks in top100

In [2]:
# import needed libraries

import lyricsgenius
import pandas as pd
import numpy as np
import getpass
from tqdm import tqdm
from bs4 import BeautifulSoup
import requests

In [7]:
# import data

top100_dr_tracks = pd.read_csv('../data/top100/top100_tracks_V3.csv')

In [8]:
top100_dr_tracks.head()

,Unnamed: 0,artist,title,label,rank,week_start,week_end,genre,year,month
0,0,nathan evans,wellerman,polydor,1,07.05.2021,13.05.2021,NaN,2021,5
1,1,cro feat. capital bra,blessed,urban,2,07.05.2021,13.05.2021,deutschrap,2021,5
2,2,jamule,liege wieder wach,life is pain,3,07.05.2021,13.05.2021,deutschrap,2021,5
3,3,riton x nightcrawlers feat. mufasa & hypeman,friday (dopamine re-edit),columbia,4,07.05.2021,13.05.2021,NaN,2021,5
4,4,lil nas x,montero (call me by your name),columbia,5,07.05.2021,13.05.2021,hiphop,2021,5


In [12]:
# remove all rows where genre NaN, christmas

top100_dr_tracks = topp100_dr_tracks.loc[topp100_dr_tracks['genre'] == 'deutschrap']
len(top100_dr_tracks)

In [18]:
# removing duplicates

top100_dr_tracks.sort_values("title", inplace = True)
top100_dr_tracks.drop_duplicates(subset ="title",
                     keep = False, inplace = True)

len(top100_dr_tracks)

2021-05-18 08:31:42,570 [88225] WARNING  py.warnings:109: [JupyterRequire] <ipython-input-18-a77bc5ab395c>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top100_dr_tracks.sort_values("title", inplace = True)

2021-05-18 08:31:42,584 [88225] WARNING  py.warnings:109: [JupyterRequire] <ipython-input-18-a77bc5ab395c>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top100_dr_tracks.drop_duplicates(subset ="title",



In [21]:
# getting a list of clean artist and track titles

artisttweaked = [a.split(" feat. ")[0].split(" x ")[0].replace(" & ", " ").replace(" / ", " ").replace("'", "").replace(" [de]", "").replace(".", " ").replace("?", "").replace("¿", "").replace(" +", "").replace(" [Live]", "") for a in top100_dr_tracks['artist']]
titletweaked = [t.split(" feat. ")[0].split(" x ")[0].replace(" & ", " ").replace(" / ", " ").replace("'", " ").replace(" [de]", "").replace(".", " ").replace("?", "").replace("¿", "").replace(" +", "").replace(" [Live]", "") for t in top100_dr_tracks['title']]

searchterms = []
for a, t in zip(artisttweaked, titletweaked):
    s = a + " " + t
    searchterms.append(s)
    
len(searchterms)

In [23]:
token = getpass.getpass()

········


In [24]:
# accessing genius API

client_access_token = token
genius = lyricsgenius.Genius(client_access_token, remove_section_headers=True,
                 skip_non_songs=True, excluded_terms=["Remix", "Live", "Edit", "Mix", "Club"])

In [25]:
# getting lyrics per track

lyrics_lst = []

for searchy in tqdm(searchterms):
    try:
        songsind = genius.search_songs(searchy)
        url = songsind['hits'][0]['result']['url']
        page = requests.get(url)
        html = BeautifulSoup(page.text, "html.parser") # Extract the page's HTML as a string

        # Scrape the song lyrics from the HTML
        lyrics = html.find("div", class_="lyrics").get_text()
        lyrics_lst.append(lyrics)
    except:
        lyrics_lst.append(np.nan)

100%|██████████| 661/661 [35:02<00:00,  3.18s/it]


In [26]:
# checking for how many tracks I did not get lyrics

lyrics_lst.count(np.nan)

345

In [48]:
top100_dr_tracks['lyrics'] = lyrics_lst

2021-05-18 09:26:49,458 [88225] WARNING  py.warnings:109: [JupyterRequire] <ipython-input-48-d95b4fa4b34f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top100_dr_tracks['lyrics'] = lyrics_lst



In [49]:
# checking data

top100_dr_tracks.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 661 entries, 1778 to 17837
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  661 non-null    int64 
 1   artist      661 non-null    object
 2   title       661 non-null    object
 3   label       659 non-null    object
 4   rank        661 non-null    int64 
 5   week_start  661 non-null    object
 6   week_end    661 non-null    object
 7   genre       661 non-null    object
 8   year        661 non-null    int64 
 9   month       661 non-null    int64 
 10  lyrics      316 non-null    object
dtypes: int64(4), object(7)
memory usage: 62.0+ KB


In [50]:
top100_dr_tracks.head()

,Unnamed: 0,artist,title,label,rank,week_start,week_end,genre,year,month,lyrics
1778,1778,kasimir1441 & chapo102,"1,50",1441,79,08.01.2021,14.01.2021,deutschrap,2021,1,"\n\n[Songtext zu „1,50“]\n\n[Intro: KASIMIR144..."
2337,2337,pa sports,100 bars final kill,life is pain,38,27.11.2020,03.12.2020,deutschrap,2020,27,\n\n[Songtext zu „100 Bars Final Kill“]\n\n[In...
11970,11970,gent feat. kurdo,100 k,alpha empire,71,25.01.2019,31.01.2019,deutschrap,2019,25,\n\n[Songtext zu „100 K“]\n\n[Intro]\nAlpha Mu...
51659,51659,anna-maria zimmermann,100.000 leuchtende sterne,electrola,60,24.06.2011,30.06.2011,deutschrap,2011,24,\n\n[Songtext zu „100.000 leuchtende Sterne“]\...
42649,42649,prinz pi feat. casper,100x,keine lieb,50,15.03.2013,21.03.2013,deutschrap,2013,15,NaN


In [46]:
# sense check

searchterms[0]

'kasimir1441 chapo102 1,50'

In [47]:
# sense check

lyrics_lst[0]

"\n\n[Songtext zu „1,50“]\n\n[Intro: KASIMIR1441, Chapo102 & Luka]\nEy, ja, das' ein Jaynbeat! Was ein Beat-Tag? Du Nutten\nDas ProblemKindMukke, Digga\nAlter, er hat mich von 'ner eins Meter fünfzig (Ey, ey) hohen Kante runter geschmissen (Ey)\n\n[Hook: KASIMIR1441]\nEy, ja, 1,50, Digga, ich fall' da nicht runter (Ey, ja, nein)\nWir sind nachts auf Henny und sie macht was mit dem Mund klar (Ah)\nIch bin im Hotel und die Schlampe hat nix drunter (Rrah)\nDie will alles haben, was sie nicht kriegt, sind die Klunker\n1,50, Digga, ich fall' da nicht runter (Bah, bah, bah, bah, bah)\nWir sind nachts auf Henny und sie macht was mit dem Mund klar (Ah)\nIch bin im Hotel und die Schlampe hat nix drunter (Nein)\nDie will alles haben, was sie nicht kriegt, sind die Klunker (Bah, bah, bah, bah, bah, bah)\n\n[Bridge: Luka]\nUnd da hat er gelacht, als ich geheult hab', also erzähl mir nichts von Zahnschmerzen!\n\n[Part: Chapo102 & KASIMIR1441]\nSchmier' mir Koka an die Zähne, doch ich habe keine Sch

In [51]:
# save data

top100_dr_tracks.to_csv('../data/lyrics/lyrics_top100_dr_tracks.csv')